\tableofcontents

\pagebreak

# Consideraciones Generales


Este informe documenta el diseño e implementación de un servicio de registro de eventos orientado a objetos en C++17. La solución permite registrar **eventos internos** y **eventos externos** (exigiendo `userId` y `deviceId`), persistirlos en disco y **consultarlos y exportarlos** en **CSV**, **JSON** o **XML**. La persistencia reusa y extiende la clase auxiliar `File` de la actividad anterior, agregando las columnas requeridas: **Fecha**, **ID** y **Tipo de Acción**. La aplicación incluye una **secuencia de prueba significativa** desde un módulo principal (main) que demuestra registro, consulta y exportación.

## Alcance


- Registro de eventos con campos: `Fecha (ISO-8601)`, `ID`, `TipoAccion`, `Nivel`, `Modulo`, `Linea`, `UserId`, `DeviceId`, `Mensaje`.
    
- Discriminación por **rango de fechas**, **tipo de acción** y **usuario**.
    
- Exportación en **CSV/JSON/XML** a partir del almacenamiento base en CSV.
    
- Uso de `File` para IO y presentación, manteniendo sus beneficios de robustez y portabilidad.
    

## Hipótesis y supuestos de diseño


- **Tiempo:** las fechas se serializan como texto ISO-8601 (`YYYY-MM-DDTHH:MM:SS`).
    
- **IDs:** cada evento recibe un identificador único (UUID o correlativo monotónico).
    
- **Entrada/salida:** el almacenamiento es **local** y secuencial (append seguro).
    
- **Errores:** líneas inválidas se descartan con conteo y mensaje; no abortan la ejecución.
    
- **Concurrencia:** versión base monohilo; si se usa multihilo, `append` se protege con mutex.
    
- **Compatibilidad:** `File` permanece como clase auxiliar; se añaden encabezados y filas con las nuevas columnas.
    

## Principios OO aplicados



- **SRP:** `RegistroEventos` encapsula la lógica de negocio; `File` encapsula IO.
    
- **DIP:** el servicio depende de una interfaz de almacenamiento (aquí materializada por `File`) y no de detalles concretos.
    
- **Bajo acoplamiento / Alta cohesión:** cada clase tiene una responsabilidad clara; la app compone los módulos.
    
- **Reutilización:** se conserva la API de `File` para lectura y presentación (CSV/JSON/XML).
    

# Esquema general de la solución




## Diseño de la solución implementada


![alt text](Imágenes/DiagramaUML.png)

- **`RegistroEventos`**: servicio que valida y registra eventos.
    
    - `logInterno(tipo, msg, modulo, linea)`
        
    - `logExterno(tipo, msg, userId, deviceId, modulo, linea)`
        
    - `query(desde, hasta, tipos, userId)`
        
    - `exportCSV/JSON/XML(filtros)`
        
- **`EventRecord`**: estructura inmutable de una fila de evento, con todas las columnas.
    
- **`File`** (auxiliar): IO de disco.
    
    - `writeHeader(...)`, `appendRow(...)`, `readAllRows()`, `toCSV/toJSON/toXML(...)`.
        

Relación: `RegistroEventos` **crea** `EventRecord` y **delegá** en `File` la persistencia y la presentación.

# Interfaces de usuario


Interfaz de **línea de comandos** mínima orientativa (puede variar con tu implementación real):

- **Registro de ejemplo (main):**
    

```bash
./app --demo               # ejecuta la secuencia de prueba significativa
```

- **Exportaciones:**
    

```bash
./app --export csv --desde 2025-10-01T00:00:00 --hasta 2025-10-07T23:59:59
./app --export json --tipo LECTURA
./app --export xml  --user u-42
```

- **Salida de ejemplo:** muestra conteo total, primeras filas y ubicación del archivo exportado.
    

# Recursos adicionales


Se emplean exclusivamente bibliotecas estándar de C++ y POSIX ya presentes en la actividad anterior: `<string>`, `<vector>`, `<fstream>`, `<stdexcept>`, `<chrono>`, `<ctime>`, `<sstream>`, `<iomanip>`, `<iostream>`, `<filesystem>`, `<algorithm>`.

# Manual de instrucciones de la aplicación

## Uso (manual breve)

### Compilación

Desde el directorio del código:

```bash
make
```

Genera el ejecutable `app`.


### Secuencia de prueba (módulo principal)

```bash
./app --demo
```

La demo:

1. Inicializa `File` y escribe cabecera si el archivo está vacío.
    
2. Registra 3 eventos **internos** con tipos distintos.
    
3. Registra 2 eventos **externos** (requiere `userId` y `deviceId`).
    
4. Realiza `query` por tipo y por usuario; imprime conteos y primeras filas.
    
5. Exporta los resultados filtrados a **CSV/JSON/XML** y muestra la ruta.
    


### Lectura/serialización (modo lectura)

```bash
./app --export csv  --tipo ACTUALIZACION
./app --export json --user u-42
./app --export xml  --desde 2025-10-01T00:00:00 --hasta 2025-10-07T23:59:59
```

Cada comando lee el CSV base con `File`, filtra y serializa al formato solicitado.


## Flujo en modo escritura (registro)


1. `RegistroEventos` valida parámetros según sea **interno** o **externo**.
    
2. Genera `fecha` (ISO-8601) e `id` único.
    
3. Construye `EventRecord` y delega `appendRow(...)` a `File`.
    
4. `File` crea cabecera y escribe la fila escapando comas y comillas.
    

## Flujo en modo lectura (consulta y exportación)

1. `File.readAllRows()` devuelve todas las filas como vectores de string.
    
2. `RegistroEventos` mapea a `EventRecord` y aplica filtros (`desde/hasta`, `tipos`, `userId`).
    
3. Exporta con `toCSV/toJSON/toXML(...)` reutilizando utilidades de `File`.
    

## Serialización y robustez



- CSV base legible y estable; JSON/XML generados a demanda.
    
- Líneas corruptas se ignoran con logging de diagnóstico.
    
- En multihilo, `append` se protege con mutex.
    


## Pruebas realizadas

- Compilación y ejecución de `--demo`.
    
- Verificación de cabecera: `Fecha,ID,TipoAccion,Nivel,Modulo,Linea,UserId,DeviceId,Mensaje`.
    
- Registro de internos/externos con y sin campos obligatorios (se espera rechazo si faltan en externos).
    
- Consultas: por rango de fechas, por tipo y por usuario.
    
- Exportaciones a CSV/JSON/XML y revisión manual de consistencia.

A continuación, se dejan capturas de pantalla como evidencia de la funcionalidad del código.

![alt text](Imágenes/Prueba1.png)

![alt text](Imágenes/Prueba2.png)

![alt text](Imágenes/Prueba3.png)

# Conclusiones



La solución satisface el requerimiento de **distinguir eventos internos y externos** con validaciones específicas, **persistir** con las nuevas columnas y **consultar/exportar** en múltiples formatos. La **separación de responsabilidades** entre `RegistroEventos` (lógica) y `File` (IO/serialización) facilita mantenimiento y futuras extensiones, por ejemplo rotación de archivos o almacenamiento alternativo (SQLite) sin reescribir la lógica de negocio.

# Referencias consultadas

- Apuntes de cátedra de POO (sección “Guía de Trabajos Prácticos”).

- Apuntes de cátedra y formato de informe de la actividad previa.

- Documentación de la [biblioteca estándar de C++](https://cplusplus.com/reference/) (streams, chrono, filesystem).
    

# Anexo - Definición de clases (resumen)

```cpp
// EventRecord.h
struct EventRecord {
    std::string fechaISO;   // "2025-10-07T16:03:12"
    std::string id;         // UUID o correlativo
    std::string tipoAccion; // "CREACION","LECTURA","ACTUALIZACION","ELIMINACION",...
    std::string nivel;      // "DEBUG","INFO","WARNING","ERROR"
    std::string modulo;
    int         linea;
    std::string userId;     // vacío si interno
    std::string deviceId;   // vacío si interno
    std::string mensaje;
};
```

```cpp
// RegistroEventos.h (fragmento)
class File; // clase auxiliar existente

class RegistroEventos {
public:
    explicit RegistroEventos(File& f);

    void logInterno (const std::string& tipo, const std::string& msg,
                     const std::string& modulo, int linea);

    void logExterno (const std::string& tipo, const std::string& msg,
                     const std::string& userId, const std::string& deviceId,
                     const std::string& modulo, int linea);

    std::vector<EventRecord> query(const std::string& desdeISO = "",
                                   const std::string& hastaISO = "",
                                   const std::vector<std::string>& tipos = {},
                                   const std::string& userId = "");

    std::string exportCSV (/* mismos filtros */);
    std::string exportJSON(/* mismos filtros */);
    std::string exportXML (/* mismos filtros */);

private:
    File& file_;
    void append(const EventRecord& e);
    std::vector<EventRecord> readAll();
};
```

```cpp
// File.h (métodos relevantes)
class File {
public:
    bool isEmpty() const;
    void writeHeader(const std::vector<std::string>& cols);
    void appendRow  (const std::vector<std::string>& cols);

    std::vector<std::vector<std::string>> readAllRows();
    std::string toCSV();
    std::string toJSON(const std::vector<std::string>& headers);
    std::string toXML (const std::vector<std::string>& headers);
};
```
